In [2]:
import json
import pandas as pd

# Openaiキーの設定

In [3]:
def get_openai_apikey():
    """get openai api key"""
    # パスは適宜読み替えて下さい
    path = r'F:\secrets\openai_apikey.txt'
    if os.path.exists(path): # localhost
        with open(path, 'r') as f:
            return f.read().replace('\n', '')
    else:
        raise Exception('no key file!')
    
import os
import openai

os.environ["OPENAI_API_KEY"] = get_openai_apikey()

In [ ]:
# パスは適宜読み替えて下さい
df = pd.read_csv('./data/pqa.csv', encoding='utf-8-sig')

# データの準備

In [9]:
def n_token(s):
    # 文字数からトークン数を推定
    # 数式の根拠は失念
    return 1.35049 * len(s) + 2.92158

df['ntokenq'] = df.q.apply(n_token)
df['ntokena'] = df.a.apply(n_token)

# question ,answerともにtoken 2000未満のレコード
dff = df[(df.ntokenq < 2000) & (df.ntokena < 2000)]

In [36]:
# 重複削除
dff = dff.drop_duplicates(subset='q').drop_duplicates(subset='a')

In [ ]:
# データの準備（jsonl）
# Data Formattingは以下参照
# https://platform.openai.com/docs/guides/fine-tuning/data-formatting
df_jsonl = pd.DataFrame({'prompt': dff.q + '\n\n###\n\n', 'completion': ' ' + dff.a + '###'})
df_jsonl.to_json('./data/qas.jsonl', orient='records', force_ascii=False, lines=True)

In [38]:
# データのアップロード
!openai tools fine_tunes.prepare_data -f ./data/qas.jsonl -q

Analyzing...

- Your file contains 4509 prompt-completion pairs
- All prompts end with suffix `\n\n###\n\n`
- All completions end with suffix `###`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "./data/qas_prepared_revised.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\n###\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 1.07 hours to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [ ]:
!openai api fine_tunes.create -t "./data/qas.jsonl"

In [4]:
def get_answer(q):
    ft_model = 'ada:ft-personal-2023-05-17-23-33-01'
    res = openai.Completion.create(
        model=ft_model, 
        prompt=q + '\n\n###\n\n', 
        max_tokens=1000,
        temperature=0.8,
        stop='###')
    a = res['choices'][0]['text']
    return a

In [3]:
q = '''28歳の男性です。高校までは明るく友人も多かったのですが、大学に進学した後、どことなく不安な感じが出現して休学しました。説明が難しいのですが、世界が不気味に変容して襲いかかってくる感じがしました。

その後、テレビを点けると出演者が突然ぎこちない話し方をします。おそらく、僕がテレビをONするとそれがスタジオに伝わる仕掛けになっているのだと思います。出演者に見られて笑われている気もします。

外出するとスパイのような人たちがうろうろしていて、自分を殺そうとしているのではないかと思いました。道を歩いていると通行人がすれ違いざまに「ばか」「消えろ」と悪口を言ってきます。夜部屋の明かりを点けると、それに合わせてお隣さんも照明を点けます。

最近気づいたことですが部屋に無数の盗聴器や監視カメラが仕掛けられています。常に見られている感覚がします。業者を呼んで取り除いてもらおうか考えています。

私は病気でしょうか。'''


print(q)
a = get_answer(q)
print('-' * 30)
print(a)

28歳の男性です。高校までは明るく友人も多かったのですが、大学に進学した後、どことなく不安な感じが出現して休学しました。説明が難しいのですが、世界が不気味に変容して襲いかかってくる感じがしました。

その後、テレビを点けると出演者が突然ぎこちない話し方をします。おそらく、僕がテレビをONするとそれがスタジオに伝わる仕掛けになっているのだと思います。出演者に見られて笑われている気もします。

外出するとスパイのような人たちがうろうろしていて、自分を殺そうとしているのではないかと思いました。道を歩いていると通行人がすれ違いざまに「ばか」「消えろ」と悪口を言ってきます。夜部屋の明かりを点けると、それに合わせてお隣さんも照明を点けます。

最近気づいたことですが部屋に無数の盗聴器や監視カメラが仕掛けられています。常に見られている感覚がします。業者を呼んで取り除いてもらおうか考えています。

私は病気でしょうか。
------------------------------
 解離性障害か、統合失調症でしょう。精神科を受診してください。


In [ ]:
q = '''30代男性です。前から調子がおかしく、「自分は素晴らしい人物だ」「皆が自分を称賛すべきだ」という考えに囚われることがあります。

また、浪費癖が時々出てしまい、ブランド物を衝動的に買い漁って後で後悔することがあります。お金を使いすぎて数百万借金しました。
その借金も返せなくなり、最終的に自己破産しました。

途端に気分が優れなくなり、一日中寝ているようになりました。食欲が減り、体重が15kg減りました。思考もまとまらず、意欲がどうしても湧きません。

私は病気でしょうか？
'''

print(q)
a = get_answer(q)
print('-' * 30)
print(a)

30代男性です。前から調子がおかしく、「自分は素晴らしい人物だ」「皆が自分を称賛すべきだ」という考えに囚われることがあります。

また、浪費癖が時々出てしまい、ブランド物を衝動的に買い漁って後で後悔することがあります。お金を使いすぎて数百万借金しました。
その借金も返せなくなり、最終的に自己破産しました。

途端に気分が優れなくなり、一日中寝ているようになりました。食欲が減り、体重が15kg減りました。思考もまとまらず、意欲がどうしても湧きません。

私は病気でしょうか？

------------------------------
 [quote]前から調子がおかしく、「自分は素晴らしい人物だ」「皆が自分を称賛すべきだ」という考えに囚われることがあります。[/quote]

      あります。統合失調症でしょう。「昔は素晴らしい人物が自分だった」と考えてしまうそこで、回復に向かうのは非常に難しいところです。

[quote]途端に気分が優れなくなり、一日中寝ているようになりました。食欲が減り、体重が15kg減りました。思考もまとまらず、意欲がどうしても湧きません。[/quote]

      このような状態では、慎重にこのようにお考えになることは稀で、統合失調症の良い時期には少なからず誤りが出てきます。結果として今の状態は、それが悪いと悪いというよりも、悪いと悪いという方がよくあることです。

[quote]私は病気でしょうか？[/quote]

      そうです、病気です。精神科を受診してください。そうすれば今の苦しさは解消に向かうと思います。





In [5]:
q = '''40代女性です。悲しい気分が止まりません。自分は価値のない人間だと思います。
5階の窓から外を見下ろして「死にたい」と感じます。気分の落ち込みや、不安感、対人恐怖に苦しんでいます。夜もよく眠れません。
私は何病でしょうか？
'''

print(q)
a = get_answer(q)
print('-' * 30)
print(a)

40代女性です。悲しい気分が止まりません。自分は価値のない人間だと思います。
5階の窓から外を見下ろして「死にたい」と感じます。気分の落ち込みや、不安感、対人恐怖に苦しんでいます。夜もよく眠れません。
私は何病でしょうか？

------------------------------
 病気かもしれないと思います。普通は夜無く、経過が曖昧ですが、病気だった場合、夜に出ることがありえます。その場合は死にたいと感じることがあります。精神科を受診してください。


In [6]:
q = '''20代女性です。人の顔が全く覚えられません。どんなに人の顔の特徴を覚えようとしてもだめです。
顔そのものはうまく覚えられないため、声や髪型、ほくろや眼鏡などで見分けています。
先日も、よく知っている人に声をかけられたのに誰だか分からずに失礼な対応をしてしまいました。
ドラマや映画も、登場人物の区別がつかずにあまり楽しめません。
私は病気なのでしょうか？
'''

print(q)
a = get_answer(q)
print('-' * 30)
print(a)

20代女性です。人の顔が全く覚えられません。どんなに人の顔の特徴を覚えようとしてもだめです。
顔そのものはうまく覚えられないため、声や髪型、ほくろや眼鏡などで見分けています。
先日も、よく知っている人に声をかけられたのに誰だか分からずに失礼な対応をしてしまいました。
ドラマや映画も、登場人物の区別がつかずにあまり楽しめません。
私は病気なのでしょうか？

------------------------------
 病気でしょう。そして、顔は病気ではないからです。どんな病気にも人根は信用できることです。
病気かどうかはわかりませんが、うれしいものです。顔は自然に消えていくことによって病気が徐々に消がされるようになります。そうすればどんな病気にも人根は信用できることが大切になります。
精神科を受診してください。
精神科で治療を受ければ今の悩みは解消に向かうのですが、それは顔の人根がない人根であることは、質問者心のことです。
この症状を本人も悩んでおられる方はすべています。
精神科を受診してください。
